# Comet

![](https://user-images.githubusercontent.com/7529846/230328046-a8b18c51-12e3-4617-9b39-97614a571a2d.png)

In this guide we will demonstrate how to track your Langchain prompts, Chains, and Agents with [Comet](https://www.comet.com/site/?utm_source=langchain&utm_medium=referral&utm_campaign=comet_notebook).


**Example Project:** [Comet with LangChain](https://www.comet.com/examples/comet-example-langchain-llm-notebook/prompts?utm_source=langchain&utm_medium=referral&utm_campaign=comet_notebook)

<img width="1280" alt="comet-langchain" src="https://user-images.githubusercontent.com/7529846/230326720-a9711435-9c6f-4edb-a707-94b67271ab25.png">


### Install Comet and Dependencies

In [ ]:
%pip install -U comet_llm "langchain>=0.1.3" "langchain-openai" openai numexpr

### Initialize Comet and Set your Credentials

In [ ]:
import comet_llm

comet_llm.init(project="comet-example-langchain-llm-notebook")

### Set OpenAI and SerpAPI credentials

You will need an [OpenAI API Key](https://platform.openai.com/account/api-keys) to run the following examples

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["OPENAI_ORGANIZATION"] = "..."

# Tracing with Comet

There are two ways to trace your LangChains executions with Comet:

1. Setting the `LANGCHAIN_COMET_TRACING` environment variable to "true". This is the recommended way.
2. Import the `CometTracer` manually and pass it explicitely.

In [ ]:
os.environ["LANGCHAIN_COMET_TRACING"] = "true"
from langchain.callbacks.tracers.comet import CometTracer

### Scenario 1: Using just an LLM

In [ ]:
from datetime import datetime

from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9, verbose=True)

for prompt in ["Tell me a joke", "Tell me a poem", "Tell me a fact"]:
    llm_result = llm.generate([prompt])
    print("LLM result", llm_result)

### Scenario 2: Using an LLM in a Chain

In [ ]:
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9, verbose=True)

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

test_prompts = [{"title": "Documentary about Bigfoot in Paris"}]
print(synopsis_chain.apply(test_prompts))

### Scenario 3: Using An Agent with Tools 

In [ ]:
from langchain.agents import initialize_agent, load_tools
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)

tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
)
agent.run(
    "What is 2 raised to .123243 power?",
)